In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import pandas as pd
from pytickersymbols import PyTickerSymbols

pd.set_option('display.max_rows', 1000)

In [30]:
browser  = webdriver.Chrome(ChromeDriverManager().install())
wait = WebDriverWait(browser, 10)

browser.get('https://advance.lexis.com/bisacademicresearchhome')
wait.until(EC.element_to_be_clickable((By.ID, 'userid'))).send_keys('tim_roethig')
wait.until(EC.element_to_be_clickable((By.ID, 'password'))).send_keys('')
wait.until(EC.element_to_be_clickable((By.ID, 'signInSbmtBtn'))).click()

[WDM] - Downloading: 100%|██████████| 8.08M/8.08M [00:01<00:00, 6.44MB/s]
/var/folders/j_/b_733cs12gg2266_mh3cd60m0000gn/T/ipykernel_20766/3870119009.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser  = webdriver.Chrome(ChromeDriverManager().install())


In [31]:
def scrape_stock_news(stock):
    browser.get('https://advance.lexis.com/bisacademicresearchhome')

    wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'recent-favorites'))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Favorisierte Filter "]'))).click()
    wait.until(EC.presence_of_element_located((By.XPATH, '//button[text()="masterarbeit "]')))
    element = browser.find_element(By.XPATH, '//button[text()="masterarbeit "]')
    browser.execute_script("arguments[0].click();", element)
    wait.until(EC.element_to_be_clickable((By.XPATH, '//button[text()="OK"]'))).click()

    browser.find_element(By.TAG_NAME, 'lng-expanding-textarea').send_keys(stock)
    browser.find_element(By.TAG_NAME, 'lng-search-button').click()

    wait.until(EC.element_to_be_clickable((By.ID, 'podfiltersbuttondatestr-news'))).click()
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'min-val')))
    for i in range(20):
        browser.find_element(By.CLASS_NAME, 'min-val').send_keys(Keys.BACK_SPACE)
    browser.find_element(By.CLASS_NAME, 'min-val').send_keys('01.01.2018')
    for i in range(20):
        browser.find_element(By.CLASS_NAME, 'max-val').send_keys(Keys.BACK_SPACE)
    browser.find_element(By.CLASS_NAME, 'max-val').send_keys('30.09.2022')
    browser.find_element(By.XPATH, '//*[@id="refine"]/div[2]/div[4]/button').click()
    time.sleep(5)

    n_res = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/header/h2/span'))).text
    n_res = re.findall(r'\((.*?)\)', n_res)[0]
    n_res = int(''.join(i for i in n_res if i.isdigit()))
    n_res = int((n_res * 0.25) // 50) + 1

    for i in range(min(n_res, 10)):
        print(i)
        flag = True
        while(flag):
            try:
                wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@aria-label="Alle Elemente auf dieser Seite auswählen"]'))).click()
                flag = False
            except:
                pass
        time.sleep(1)
        flag = 0
        while(flag < 5):
            try:
                flag += 1
                wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@aria-label="Weiter"]'))).click()
                flag = 10
                time.sleep(1)
            except:
                pass
        time.sleep(5)
        wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@aria-label="Teilen mit Cloud-Speicher"]'))).click()
        wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@aria-label="An Dropbox senden (aktuelle Einstellungen)"]'))).click()
        for _ in range(100):
            try:
                wait.until(EC.element_to_be_clickable((By.ID, 'FileName'))).send_keys(Keys.BACK_SPACE)
            except:
                pass
        file_name = stock.lower().replace(' ', '_').replace('ä', 'ae').replace('ü', 'ue').replace('ö', 'oe').replace('+', '').replace('.', '').replace('ó', 'o') + f'_{i}'
        wait.until(EC.element_to_be_clickable((By.ID, 'FileName'))).send_keys(file_name)
        flag = True
        while(flag):
            try:
                wait.until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Senden"]'))).click()
                flag = False
            except:
                pass
        wait.until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Fortfahren"]'))).click()
        time.sleep(20+i)

In [33]:
stock_data = PyTickerSymbols()
sdax = list(stock_data.get_stocks_by_index('SDAX'))
mdax = list(stock_data.get_stocks_by_index('MDAX'))

stocks = list()
stocks.extend(sdax)
stocks.extend(mdax)

stocks = pd.DataFrame(stocks)
stocks['name'] = stocks['name'].str.lower()
stocks['name'] = stocks['name'].str.replace(' & co. kgaa', '')
stocks['name'] = stocks['name'].str.replace('&co.kgaa', '')
stocks['name'] = stocks['name'].str.replace(' & co. kg', '')
stocks['name'] = stocks['name'].str.replace('vz', '')
stocks['name'] = stocks['name'].str.replace('vna', '')
stocks.loc[stocks['name']=='strör se', 'name'] = 'ströer se'

stocks = stocks[['name', 'symbol']].drop_duplicates().sort_values('name').reset_index(drop=True)
stock_list = stocks['name'].to_list()

for stock in stock_list[109:]:
    print(stock)
    scrape_stock_news(stock)
    time.sleep(10)

/var/folders/j_/b_733cs12gg2266_mh3cd60m0000gn/T/ipykernel_20766/1626202529.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  stocks['name'] = stocks['name'].str.replace(' & co. kgaa', '')
/var/folders/j_/b_733cs12gg2266_mh3cd60m0000gn/T/ipykernel_20766/1626202529.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  stocks['name'] = stocks['name'].str.replace('&co.kgaa', '')
/var/folders/j_/b_733cs12gg2266_mh3cd60m0000gn/T/ipykernel_20766/1626202529.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  stocks['name'] = stocks['name'].str.replace(' & co. kg', '')


uniper se
0
1
2
3
4
united internet ag
0
1
2
vantage towers ag
0
varta ag
0
1
verbio vereinigte bioenergie ag
0
1
wacker chemie ag
0
1
2
3
4
5
6
7
8
9
wacker neuson se
0
1
2
westwing group ag
0
xing ag
0
zeal network se
0
1
zooplus ag
0
1


In [7]:
min(10, 1)

1

In [ ]:
    flag = True
    while(flag):
        try:
            wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@aria-label="Herunterladen (mit den aktuellen Einstellungen)"]'))).click()
            flag = False
        except:
            pass
    for _ in range(100):
        try:
            wait.until(EC.element_to_be_clickable((By.ID, 'FileName'))).send_keys(Keys.BACK_SPACE)
        except:
            pass
    wait.until(EC.element_to_be_clickable((By.ID, 'FileName'))).send_keys(stock.lower().replace(' ', '_') + f'_{i}')
    flag = True
    while(flag):
        try:
            wait.until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Herunterladen"]'))).click()
            flag = False
        except:
            pass
    time.sleep(10)